# Group Proposal

add intro paragraph talking about tennis breifly

Target Question: Does a player handedness have influence on their ranking i.e will a left handed player be more likely to be in the top 10 rankings? 

In [4]:
library(tidyverse)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [12]:
player_data <- read_csv("player_info.csv") |>
        select(height_cm, weight, primaryPosition) 
player_data


Rows: 3925 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): firstName, lastName, nationality, birthCity, primaryPosition, birt...
dbl  (3): player_id, height_cm, weight
dttm (1): birthDate

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


height_cm,weight,primaryPosition
<dbl>,<dbl>,<chr>
185.42,207,RW
185.42,210,D
180.34,180,D
177.80,192,D
190.50,220,D
187.96,206,RW
180.34,185,C
187.96,216,G
175.26,173,LW
